In [3]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [8]:
import pandas as pd
import numpy as np

# Read the data file
df = pd.read_csv('../data/IndeedJobScraperIntern.csv')

# Display basic info about the dataset
print(f"Dataset shape: {df.shape}")
print(f"\nColumn names: {df.columns.tolist()}")

# Check the Posted_Date column
print(f"\nPosted_Date column info:")
print(df['Posted_Date'].describe())

# Convert Posted_Date to datetime
df['Posted_Date'] = pd.to_datetime(df['Posted_Date'], errors='coerce')

# Find the range (min and max)
min_date = df['Posted_Date'].min()
max_date = df['Posted_Date'].max()

print(f"\nDate Range in Posted_Date column:")
print(f"Minimum (Earliest) Date: {min_date}")
print(f"Maximum (Latest) Date: {max_date}")
print(f"Date Range: {max_date - min_date}")

# Check for any null values
null_count = df['Posted_Date'].isna().sum()
print(f"\nNull values in Posted_Date: {null_count}")


Dataset shape: (474, 20)

Column names: ['Site', 'Search_Term', 'Search_Location', 'URL_Input', 'Total_Result', 'Job_Title', 'Job_ID', 'Job_URL', 'Job_Type', 'Salary', 'Location', 'Full_Description', 'Company_Name', 'Company_URL', 'Company_Rating', 'Company_Review_Count', 'Posted_Date', 'isExpired', 'Valid_Through', 'Apply_Link']

Posted_Date column info:
count                          474
unique                         473
top       2025-08-27T01:19:50.177Z
freq                             2
Name: Posted_Date, dtype: object

Date Range in Posted_Date column:
Minimum (Earliest) Date: 2024-11-22 17:12:56.219000+00:00
Maximum (Latest) Date: 2025-12-17 12:53:19.602000+00:00
Date Range: 389 days 19:40:23.383000

Null values in Posted_Date: 0


In [10]:
import re

# Define all skills by category
skills = {
    'Programming': ['Python', 'SQL', 'R', 'Julia', 'Rust', 'Scala', 'Java', 'C++', 'C#', 'Kotlin', 'Go', 'Pyspark', 
                    'JavaScript', 'TypeScript'],
    'Mathematics & Statistics': ['Linear Algebra', 'Calculus', 'Probability', 'Statistics', 'Bayesian Inference', 
                                 'Optimization', 'Resampling', 'Cross-validation', 'Bootstrapping', 'Matrix operations', 
                                 'Partial derivatives'],
    'Machine Learning': ['Supervised Learning', 'Unsupervised Learning', 'Deep Learning', 'Reinforcement Learning', 
                         'Reinforcement Learning (RL)', 'RL', 'XGBoost', 'CatBoost', 'Random Forest', 'CNN', 'RNN', 
                         'Transformers', 'SVM'],
    'AI Agents & Generative AI': ['AI Agents', 'Agentic AI', 'Multi-agent systems', 'LLMs', 'NLP', 'Prompt Engineering', 
                                  'Retrieval-Augmented Generation', 'RAG', 'Model Context Protocol', 'MCP', 
                                  'Agent-to-Agent', 'A2A', 'A2A protocols', 'Fine-tuning', 'Token optimization', 
                                  'Context management', 'Agent hierarchies'],
    'Agentic Frameworks & Tools': ['LangChain', 'CrewAI', 'AutoGen', 'LangGraph', 'AutoGPT', 'LlamaIndex', 
                                   'Microsoft Semantic Kernel', 'OpenAI Swarm', 'Vercel AI SDK', 'DSPy', 
                                   'Amazon Bedrock AgentCore', 'Hugging Face Transformers Agents', 'Langflow'],
    'Causal Inference & Experimentation': ['A/B Testing', 'Geo Experiments', 'Difference-in-Differences', 'DiD', 
                                            'Synthetic Control', 'Instrumental Variables', 'Propensity Score Matching', 
                                            'Uplift Modeling', 'Quasi-experimental design'],
    'Data Engineering & MLOps': ['ETL', 'ELT', 'ETL/ELT', 'Docker', 'Kubernetes', 'Apache Spark', 'Spark', 'Kafka', 
                                 'Airflow', 'dbt', 'MLflow', 'CI/CD', 'MLOps', 'Model monitoring', 'Observability', 
                                 'Tracing', 'AI Guardrails', 'Red-teaming'],
    'Database Systems': ['PostgreSQL', 'MySQL', 'Agentic Postgres', 'NoSQL', 'MongoDB', 'Cassandra', 'Redis', 'Neo4j', 
                        'Snowflake', 'BigQuery', 'Vector Databases', 'Pinecone', 'Milvus', 'Weaviate', 'Qdrant', 
                        'Chroma', 'FAISS'],
    'Visualization & BI': ['Tableau', 'Power BI', 'Looker', 'D3.js', 'Plotly', 'Dash', 'Matplotlib', 'Seaborn', 
                          'Streamlit'],
    'Professional Skills & Ethics': ['Data Storytelling', 'Responsible AI', 'Ethical Governance', 'GDPR 2.0', 
                                    'AI Act compliance', 'Domain Expertise', 'Cross-functional collaboration', 
                                    'Problem-solving', 'Critical Thinking']
}

# Flatten all skills into a single list with their categories
all_skills = []
for category, skill_list in skills.items():
    for skill in skill_list:
        all_skills.append((skill, category))

# Function to count skill occurrences in text (case-insensitive, word boundary aware)
def count_skill_frequency(df, skills_list):
    skill_counts = {}
    total_jobs = len(df)
    
    # Get all Full_Description texts
    descriptions = df['Full_Description'].fillna('').astype(str)
    
    for skill, category in skills_list:
        # Create regex pattern with word boundaries for exact matches
        # Handle special characters in skill names
        pattern = r'\b' + re.escape(skill) + r'\b'
        count = descriptions.str.contains(pattern, case=False, na=False, regex=True).sum()
        skill_counts[skill] = {
            'count': count,
            'percentage': (count / total_jobs * 100) if total_jobs > 0 else 0,
            'category': category
        }
    
    return skill_counts

# Calculate skill frequencies
skill_frequencies = count_skill_frequency(df, all_skills)

# Create a DataFrame for better visualization
skill_df = pd.DataFrame([
    {
        'Skill': skill,
        'Category': info['category'],
        'Count': info['count'],
        'Percentage': info['percentage']
    }
    for skill, info in skill_frequencies.items()
])

# Sort by count descending
skill_df = skill_df.sort_values('Count', ascending=False)

# Display results by category
print("=" * 80)
print("SKILL FREQUENCY ANALYSIS")
print("=" * 80)
print(f"\nTotal jobs analyzed: {len(df)}")
print(f"Total unique skills searched: {len(all_skills)}\n")

# Display top skills overall
print("\n" + "=" * 80)
print("TOP 20 MOST FREQUENT SKILLS")
print("=" * 80)
print(skill_df.head(20).to_string(index=False))

# Display by category
print("\n" + "=" * 80)
print("SKILL FREQUENCIES BY CATEGORY")
print("=" * 80)

for category in skills.keys():
    category_skills = skill_df[skill_df['Category'] == category].sort_values('Count', ascending=False)
    if len(category_skills) > 0:
        print(f"\n{category}:")
        print("-" * 80)
        print(category_skills.to_string(index=False))
        print(f"\nTotal mentions in {category}: {category_skills['Count'].sum()}")
        print(f"Average percentage per skill: {category_skills['Percentage'].mean():.2f}%")

# Summary statistics
print("\n" + "=" * 80)
print("SUMMARY STATISTICS")
print("=" * 80)
print(f"\nTotal skill mentions: {skill_df['Count'].sum()}")
print(f"Average mentions per skill: {skill_df['Count'].mean():.2f}")
print(f"Median mentions per skill: {skill_df['Count'].median():.2f}")
print(f"Skills with 0 mentions: {(skill_df['Count'] == 0).sum()}")
print(f"Skills with 10+ mentions: {(skill_df['Count'] >= 10).sum()}")
print(f"Skills with 50+ mentions: {(skill_df['Count'] >= 50).sum()}")
print(f"Skills with 100+ mentions: {(skill_df['Count'] >= 100).sum()}")


SKILL FREQUENCY ANALYSIS

Total jobs analyzed: 474
Total unique skills searched: 130


TOP 20 MOST FREQUENT SKILLS
                 Skill                     Category  Count  Percentage
                Python                  Programming    360   75.949367
            Statistics     Mathematics & Statistics    163   34.388186
                   SQL                  Programming    161   33.966245
       Problem-solving Professional Skills & Ethics    146   30.801688
                     R                  Programming    136   28.691983
          Optimization     Mathematics & Statistics     97   20.464135
         Deep Learning             Machine Learning     93   19.620253
                  LLMs    AI Agents & Generative AI     77   16.244726
                   NLP    AI Agents & Generative AI     75   15.822785
                  Java                  Programming     57   12.025316
              Power BI           Visualization & BI     50   10.548523
               Tableau           